In [104]:
import numpy as np
import matplotlib
import nltk
import os
import re

from utils import *

%matplotlib inline

In [7]:
# define experiment
exp='/exp_1a'

In [84]:
# load dataset into memory 
# get both w2i, i2w, w2freq dictionaries and lists of input and output sentences respectively
cmd_vocab, w2i_cmds, i2w_cmds, cmds, act_vocab, w2i_acts, i2w_acts, acts = load_dataset(exp=exp, split='/train')

In [71]:
train_commands = s2i(cmds, w2i_cmds)
train_actions = s2i(acts, w2i_acts)

In [72]:
train_commands

array([array([ 9,  5,  1,  3,  8, 13,  5,  1,  4]),
       array([11,  5,  2,  7, 12,  1]), array([12,  7, 11,  6,  1,  3]),
       ..., array([11,  1,  4,  8, 13,  6,  1,  3]),
       array([11,  5,  1,  4,  8, 12,  3]),
       array([10,  5,  1,  3,  7,  9,  6,  1,  3])], dtype=object)